# Paquetes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import transformers
import os

# Parametros

In [2]:
# MODEL

_LANGUAGE_         = 'es'
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = None
_MAX_SEQ_LEN_      = 128
_OUTPUT_DIR_       = 'fine_tuning_checkPoints'
_LOGGING_STEPS_    = 50


# TRAIN

_NO_GPUS_          = 5
_BATCH_SIZE_       = 100
_EPOCHS_           = 20

# Dataset

In [3]:
# LABEL DICTONARIES -----------------------------------------------------------------------

gender_dict    = {'female': 0, 
                  'male':   1}

varietyEN_dict = {'australia'    : 0,
                  'canada'       : 1,
                  'gran britain' : 2,
                  'ireland'      : 3,
                  'new zeland'   : 4,
                  'united states': 5}

varietyES_dict = {'argentina': 0,
                  'chile'    : 1,
                  'colombia' : 2,
                  'mexico'   : 3,
                  'peru'     : 4,
                  'spain'    : 5,
                  'venezuela': 6}    

In [4]:
# SET LANGUAGE DIRECTORY

if _LANGUAGE_ == 'en':
    variety_dict = varietyEN_dict

elif _LANGUAGE_ == 'es':
    variety_dict = varietyES_dict

In [5]:
# SET LANGUAGE TOKENIZER

from transformers import AutoTokenizer

if _LANGUAGE_ == 'es':
    tokenizer = AutoTokenizer.from_pretrained('pysentimiento/robertuito-base-cased')
    
vocab = tokenizer.get_vocab()

In [6]:
# CREATE ONE INSTANCE PER DATA SPLIT

from DatasetPAN17 import BasePAN17, DatasetPAN17

baseTrain = BasePAN17(Dir              = 'data',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_)

baseTest  = BasePAN17(Dir              = 'data',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done

Total Instances: 84000


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done

Total Instances: 56000



In [7]:
# CREATE DATA LOADER FOR EVERY TASK

tasks = ['gender', 'variety']

dataset_dict = {}
for task in tasks:
    dataset_dict[task] = DatasetPAN17(Base_Dataset = baseTrain, label = task)

# Model

In [8]:
from transformers import RobertaConfig, RobertaModelWithHeads

num_v           = len(baseTrain.variety_dict)
num_labels_dict = {"gender": 2, "variety": num_v,}
device = "cuda:0" if torch.cuda.is_available() else "cpu"

if _LANGUAGE_ == 'es':
    
    models = {}
    
    for task in tasks:
    
        config = RobertaConfig.from_pretrained(
            "pysentimiento/robertuito-base-cased",
            num_labels = num_labels_dict[task],
        )

        models[task] = RobertaModelWithHeads.from_pretrained(
            "pysentimiento/robertuito-base-cased",
            config = config,
        )
        
        models[task].add_classification_head(
            head_name    = task,
            num_labels   = num_labels_dict[task],
        )
        
        models[task].to(device)

/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/adapters/models/roberta.py:250: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/adapters/models/roberta.py:228: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at pysentimiento/robertuito-base-cased were not used when initializing RobertaModelWithHeads: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (

# Training

In [9]:
from Training import train_models

train_models(models        = models,
             dataset_dict  = dataset_dict,
             epochs        = _EPOCHS_,
             batch_size    = _BATCH_SIZE_,
             no_gpus       = _NO_GPUS_,
             output_dir    = _OUTPUT_DIR_, 
             logging_steps = _LOGGING_STEPS_)

/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 84000
  Num Epochs = 20
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 500
  Gradient Accumulation steps = 1
  Total optimization steps = 3360
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
50,0.593800
100,0.528000
150,0.495800
200,0.425500
250,0.361700
300,0.353100
350,0.321200
400,0.232200
450,0.240600
500,0.243200


Saving model checkpoint to fine_tuning_checkPoints/gender/checkpoint-500
Configuration saved in fine_tuning_checkPoints/gender/checkpoint-500/config.json
Model weights saved in fine_tuning_checkPoints/gender/checkpoint-500/pytorch_model.bin
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to fine_tuning_checkPoints/gender/checkpoint-1000
Configuration saved in fine_tuning_checkPoints/gender/checkpoint-1000/config.json
Model weights saved in fine_tuning_checkPoints/gender/checkpoint-1000/pytorch_model.bin
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; w

Step,Training Loss
50,0.967900
100,0.928800
150,1.074800
200,0.620900
250,0.557100
300,0.532000
350,0.502900
400,0.423700
450,0.354200
500,0.367300


Saving model checkpoint to fine_tuning_checkPoints/variety/checkpoint-500
Configuration saved in fine_tuning_checkPoints/variety/checkpoint-500/config.json
Model weights saved in fine_tuning_checkPoints/variety/checkpoint-500/pytorch_model.bin
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to fine_tuning_checkPoints/variety/checkpoint-1000
Configuration saved in fine_tuning_checkPoints/variety/checkpoint-1000/config.json
Model weights saved in fine_tuning_checkPoints/variety/checkpoint-1000/pytorch_model.bin
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scal

# Testing

In [10]:
from TestingPAN17 import test_models

accuracy = test_models(models, baseTest)

100%|███████████████████████████████████████| 2800/2800 [15:30<00:00,  3.01it/s]


In [11]:
accuracy

{'gender': 0.8289285714285715,
 'variety': 0.9435714285714286,
 'joint': 0.7882142857142858}